In [1]:
import pyodbc
import pandas as pd
import adal

In [2]:
# Connecting to DB TEST
server ="test41-eus2-ins-sqlepi-orderinteraction.database.windows.net"
database="FreshRWOAPIBufferTestDB" 
username="abdus.samad@kroger.com"
Authentication='ActiveDirectoryInteractive'
src_conn = pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};SERVER="
        + server
        + ";DATABASE="
        + database
        + ";UID="
        + username
        + ";AUTHENTICATION="
        + Authentication
    )
print("\n\nConnected to Database.")



Connected to Database.


In [3]:
# Define columns to compare
cols_to_compare = ["TRIM(C.CON_DSC_TX) AS ItemDescription",                   
                   "D.CON_TYP_CD AS UPC_Type",                   
                   "CASE WHEN I.FYT_REC_DPT_CD IN ('15','56','73') THEN '10' ELSE I.FYT_REC_DPT_CD END AS DepartmentNumber",                    
                   "H.FYT_SUB_DPT_CD AS MerchandisingFamilyNumber",                    
                   "G.FYT_COM_CD AS MerchandisingCategoryNumber",                    
                   "E.CAS_NET_WGT_AM AS CaseSize",                   
                   "E.SHP_PAK_QY AS Quantity",
                   "B.QPS_SCN_CD AS ScanCode",]


In [4]:
# Define table names
table_names =  ['[dbo].[NRM_CON_UPC_STO_PRC_034]'
,'[dbo].[NRM_CON_UPC_STO_PRC_014]'
,'[dbo].[NRM_CON_UPC_STO_PRC_011]'
,'[dbo].[NRM_CON_UPC_STO_PRC_021]'
,'[dbo].[NRM_CON_UPC_STO_PRC_016]'
,'[dbo].[NRM_CON_UPC_STO_PRC_018]'
,'[dbo].[NRM_CON_UPC_STO_PRC_024]'
,'[dbo].[NRM_CON_UPC_STO_PRC_025]'
,'[dbo].[NRM_CON_UPC_STO_PRC_026]'
,'[dbo].[NRM_CON_UPC_STO_PRC_029]'
,'[dbo].[NRM_CON_UPC_STO_PRC_035]'
,'[dbo].[NRM_CON_UPC_STO_PRC_531]'
,'[dbo].[NRM_CON_UPC_STO_PRC_534]'
,'[dbo].[NRM_CON_UPC_STO_PRC_615]'
,'[dbo].[NRM_CON_UPC_STO_PRC_620]'
,'[dbo].[NRM_CON_UPC_STO_PRC_660]'
,'[dbo].[NRM_CON_UPC_STO_PRC_701]'
,'[dbo].[NRM_CON_UPC_STO_PRC_703]'
,'[dbo].[NRM_CON_UPC_STO_PRC_704]'
,'[dbo].[NRM_CON_UPC_STO_PRC_705]'
,'[dbo].[NRM_CON_UPC_STO_PRC_706]'
,'[dbo].[NRM_CON_UPC_STO_PRC_708]']

In [5]:
import warnings

warnings.filterwarnings('ignore')

In [6]:
#Run dataset for only 122 mismatch Items. so that dataset is sorted and in one place.
common_upcs = ['0003150659502' ,'0001111060263' ,'0007210625658' ,'0002370001511' ,'0004172302341' ,'0004660002941' ,'0003150662642' ,'0003150656027' ,'0001111095557' ,'0002370051555' ,'0003150659302' ,'0003560005154' ,'0003907368522' ,'0004660003121' ,'0069511960009' ,'0001115052625' ,'0003150662303' ,'0001111099415' ,'0007030903252' ,'0004660002901' ,'0007111700188' ,'0003150652822' ,'0078620261520' ,'0078620261519' ,'0001111060365' ,'0003150656001' ,'0003318601110' ,'0001111099498' ,'0004660004120' ,'0003907304055' ,'0001115078596' ,'0001115004848' ,'0064523001106' ,'0069511946505' ,'0004530035009' ,'0001115078595' ,'0003150653302' ,'0007597110901' ,'0001111019000' ,'0001111058675' ,'0002540004133' ,'0002781530249' ,'0003150663901' ,'0069511940419' ,'0069511934660' ,'0001111018103' ,'0001111060328' ,'0003150669802' ,'0007111714134' ,'0007030901112' ,'0001111037260' ,'0003907301907' ,'0001111061627' ,'0003907304050' ,'0003907302161' ,'0001111060271' ,'0004530018588' ,'0001111060274' ,'0002265532007' ,'0003150657202' ,'0001115078640' ,'0003150662402' ,'0003150661602' ,'0007597112980' ,'0001111095558' ,'0078620262750' ,'0001111000041' ,'0003150666302' ,'0002370057085' ,'0001111074214' ,'0001111099501' ,'0003907304208' ,'0003150642002' ,'0007320241518' ,'0008837403024' ,'0003150613102' ,'0004660002515' ,'0003234512064' ,'0078620262907' ,'0007320241519' ,'0002781559709' ,'0003150663502' ,'0084013430706' ,'0003150676401' ,'0001111007037' ,'0001111092012' ,'0007750901250' ,'0001111060540' ,'0001115078630' ,'0003150663512' ,'0078620261518' ,'0003150656703' ,'0001111011213' ,'0004320017175' ,'0001111058362' ,'0002781530252' ,'0003150661802' ,'0003150657401' ,'0069511960042' ,'0003150663102' ,'0002265527312' ,'0000205591902' ,'0001111009763' ,'0003861636017' ,'0001111074079' ,'0001111052230' ,'0004660002471' ,'0002781500868' ,'0002265529032' ,'0007030901122' ,'0002781530253' ,'0007111711441' ,'0001111018106' ,'0001111003861' ,'0001111060734' ,'0001111058667' ,'0001111095559' ,'0001111099024' ,'0085466100130' ,'0003150677001' ,'0007283002501' ,'0007111711439']

In [7]:
print("Number of common UPCs: ", len(common_upcs))
#common_upcs_15 = common_upcs[:15]
#print(common_upcs_15)

Number of common UPCs:  122


In [8]:
# Convert common_upcs to a list of quoted strings
quoted_upcs = [f"'{upc}'" for upc in common_upcs]

In [9]:
# Define empty list to store dataframes for each table --- ADD TABLE NAME TO RECORDS FOR ANALYSIS
df_list = []
# Loop through each table to fetch data for common UPCs
for table in table_names:
    query = f"""
        SELECT DISTINCT {table}.CON_UPC_NO, {','.join(cols_to_compare)} FROM {table} INNER JOIN NRM_BAS_UPC B ON {table}.BAS_UPC_ID = B.BAS_UPC_ID INNER JOIN NRM_CON_UPC C ON B.CON_UPC_NO = C.CON_UPC_NO INNER JOIN NRM_CAS_CON_UPC D ON C.CON_UPC_ID = D.CON_UPC_ID INNER JOIN NRM_CAS_UPC E ON D.CAS_UPC_ID = E.CAS_UPC_ID INNER JOIN NRM_FYT_SCM F ON E.FYT_SUB_COM_ID = F.FYT_SUB_COM_ID INNER JOIN NRM_FYT_COM G ON G.FYT_COM_ID = F.FYT_COM_ID INNER JOIN NRM_FYT_SUB_DPT H ON H.FYT_SUB_DPT_ID = G.FYT_SUB_DPT_ID INNER JOIN NRM_FYT_REC_DPT I ON I.FYT_REC_DPT_ID = H.FYT_REC_DPT_ID WHERE H.FYT_SUB_DPT_CD IN ('15','56','73') AND D.CON_TYP_CD = '1' AND B.QPS_SCN_CD IN ('W', 'S', 'C') AND {table}.CON_ITM_STU_CD IN ('I', 'A') AND I.FYT_REC_DPT_CD IN ('15','56','73') AND G.FYT_COM_CD IN ('450','451','463','456','457') 
        AND {table}.CON_UPC_NO IN ({','.join(quoted_upcs)})
        """
    df = pd.read_sql(query, src_conn)
    df['table_names'] = table
    df_list.append(df)
    print("Query run complete!!")

Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!
Query run complete!!


In [10]:
# Merge dataframes on UPC column
df_merged = pd.concat(df_list).reset_index(drop=True)
# Group by UPC and compare values
df_grouped = df_merged.groupby('CON_UPC_NO',).nunique().max(axis=1)[1:]

In [11]:
# Combine all dataframes into a single dataframe
df = pd.concat(df_list)

In [16]:
# Sort the final dataframe by 'CON_UPC_NO'
df = df.sort_values(['CON_UPC_NO','table_names'])

In [17]:
df.to_csv("C:\\Users\\AS81970\\Desktop\\output_with_table.csv", index=False)

In [14]:
# Group the data by UPC and compare the values for each column
grouped = df.groupby('CON_UPC_NO',)
mismatches = []
for upc, group in grouped:
    ref_row = group.iloc[0]
    for _, row in group.iterrows():
        if not ref_row.equals(row.fillna(ref_row)):
            mismatches.append(upc)

if not mismatches:
    print("All data matches.")
else:
    print("Data mismatches found:")
    for upc in set(mismatches):
        print(f"UPC {upc} has mismatches in table.")


Data mismatches found:
UPC 0078620262907 has mismatches in table.
UPC 0001111095559 has mismatches in table.
UPC 0003150662303 has mismatches in table.
UPC 0007320241519 has mismatches in table.
UPC 0001111058362 has mismatches in table.
UPC 0004660002515 has mismatches in table.
UPC 0001111058667 has mismatches in table.
UPC 0007111700188 has mismatches in table.
UPC 0003150657401 has mismatches in table.
UPC 0000205591902 has mismatches in table.
UPC 0078620261519 has mismatches in table.
UPC 0008837403024 has mismatches in table.
UPC 0003150656703 has mismatches in table.
UPC 0001115052625 has mismatches in table.
UPC 0069511960009 has mismatches in table.
UPC 0001115078596 has mismatches in table.
UPC 0003150656027 has mismatches in table.
UPC 0003150656001 has mismatches in table.
UPC 0001115078630 has mismatches in table.
UPC 0001115004848 has mismatches in table.
UPC 0003907304055 has mismatches in table.
UPC 0003150666302 has mismatches in table.
UPC 0004320017175 has mismatche